<a href="https://colab.research.google.com/github/vermaAstra/generative-ai/blob/main/agents/geminidataanalytics/colabs/geminidataanalytics_public_http_demo_%7C_vPublished.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Copyright 2025 Google LLC.
# SPDX-License-Identifier: Apache-2.0

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Background and Overview
The **Conversational Analytics API** lets you chat with your BigQuery or Looker data anywhere, including embedded Looker dashboards, Slack and other chat apps, or even your own web applications. Your team members can get answers where they need them, when they need them, in the applications they use every day. You can find the [Colab example with the Python SDK here](https://colab.sandbox.google.com/drive/1UF1I4ZGU662R74vbNwk-wtmVWpQl107h?resourcekey=0-vQ8nXlhIpj87G65FbkjG8A#scrollTo=gBy8t2OiR73i).

Please provide feedback to conversational-analytics-api-feedback@google.com
<br>
### This notebook will help you
1. Authenticate to GCP
2. Add data
3. Perform agent operations (create, list, get, delete)
4. Manage conversations (create, list, get)
5. Ask questions with your agent


## Disclaimers
- Please **make a copy** of this notebook. **Do not edit it directly**
- This is a **Pre-GA** product. See [documentation](https://cloud.google.com/gemini/docs/conversational-analytics-api/overview) for more details.
- This product is currently under allowlist and not yet publicly available. Please contact your account team if you have questions.


## API Enablement

**Please fill in the billing_project form field with your own Google Cloud project.  The project must have the following APIs enabled:**
-  [cloudaicompanion API](https://console.cloud.google.com/apis/library/cloudaicompanion.googleapis.com)
-  [Gemini Data Analytics API](https://console.cloud.google.com/apis/library/geminidataanalytics.googleapis.com)
-  [BQ API](https://console.cloud.google.com/marketplace/product/google/bigquery.googleapis.com)
-  [Dataform API](https://console.cloud.google.com/apis/library/dataform.googleapis.com)
- [Vertex AI API](https://console.cloud.google.com/apis/library/aiplatform.googleapis.com)

You may pass in any BigQuery project/dataset/table for which you have read permissions.


# Auth and Imports

In [ ]:
from pygments import highlight, lexers, formatters
import pandas as pd
import json as json_lib
import json
import altair as alt
import IPython
from IPython.display import display, HTML
import requests
import google.auth
from google.auth.transport.requests import Request

from google.colab import auth
auth.authenticate_user()

access_token = !gcloud auth application-default print-access-token
headers = {
    "Authorization": f"Bearer {access_token[0]}",
    "Content-Type": "application/json",
}

# Datasource and Billing project


*   Define your billing project within GCP
*   Link your agent to data sources (BigQuery, Looker, Looker Studio)



In [ ]:
# @title Set Datasource and billing project

billing_project = "cloud-raspberry-e2e-testing" # @param {type:"string"}

location = "global"

# provide critical context for your Conversational Analytics Agent here
system_instruction = "Think like an Analyst"


# BigQuery datasource
bigquery_data_sources = {
    "bq": {
      "tableReferences": [
        {
          "projectId": "bigquery-public-data",
          "datasetId": "faa",
          "tableId": "us_airports"
        }
      ]
    }
}

# Looker datasource
# When connecting to the Looker datasource, you can authenticate using either:
# 1. client_id and client_secret
# 2. access_token
looker_credentials = {
    "oauth": {
        "secret": {
          "client_id": "Enter your Client ID Here",
          "client_secret": "Enter your Client Secret Here",
        }
    }
}
# Uncomment this and comment out the above one if you are using access_token for authentication
# looker_credentials = {
#     "oauth": {
#         "token": {
#           "access_token": "your_looker_access_token",
#         }
#     }
# }
looker_data_source = {
    "looker": {
      "explore_references": {
          "looker_instance_uri": "https://my_company.looker.com",
          "lookml_model": "my_model",
          "explore": "my_explore",
      },
      # NOTE -
      # 1. Do not pass looker credentials during create data agent.
      # 2. Uncomment the below line only when running the method [Stateless Chat using inline context]
      # "credentials": looker_credentials
    }
}



# Looker Studio datasource
looker_studio_data_source = {
    "studio":{
        "studio_references":
         [
            {
              "datasource_id": "your_studio_datasource_id"
            }
         ]
    }
}


In [ ]:
# @title Enviroment

# ENVIRONMENT OPTIONS: prod, autopush, staging
environment = "prod" # @param {type:"string"}

base_url = "https://geminidataanalytics.googleapis.com"

if (environment == "autopush"):
  base_url = "https://autopush-geminidataanalytics.sandbox.googleapis.com"
elif (environment == "staging"):
  base_url = "https://staging-geminidataanalytics.sandbox.googleapis.com"
else:
  base_url = "https://geminidataanalytics.googleapis.com"


#Data Agent Services

In [ ]:
# @title Create Data Agent

data_agent_id = "data_agent_1" # @param {type:"string"}

data_agent_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/dataAgents"

data_agent_payload = {
      "name": f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}", #Optional
      "description": "This is the description of data_agent.", #Optional

      "data_analytics_agent": {
          "published_context": {
              "datasource_references": bigquery_data_sources,
              "system_instruction": system_instruction,
              "options": {
                  "analysis": {
                      # Optional - if wanting to use advanced analysis with python
                      "python": {
                          "enabled": True
                      }
                  }
              }
          }
      }
  }

params = {"data_agent_id": data_agent_id} #Optional

data_agent_response = requests.post(
    data_agent_url, params=params, json=data_agent_payload, headers=headers
)

if data_agent_response.status_code == 200:
    print("Data Agent created successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error creating Data Agent: {data_agent_response.status_code}")
    print(data_agent_response.text)

In [ ]:
# @title Get Data Agent

data_agent_id = "data_agent_1" # @param {type:"string"}

data_agent_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"

data_agent_response = requests.get(
    data_agent_url, headers=headers
)

if data_agent_response.status_code == 200:
    print("Fetched Data Agent successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error: {data_agent_response.status_code}")
    print(data_agent_response.text)


In [ ]:
# @title List Data Agent

data_agent_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/dataAgents"

data_agent_response = requests.get(
    data_agent_url, headers=headers
)

if data_agent_response.status_code == 200:
    print("Data Agent Listed successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error Listing Data Agent: {data_agent_response.status_code}")
    print(data_agent_response.text)

In [ ]:
# @title Update Data Agent

data_agent_id = "data_agent_1" # @param {type:"string"}

data_agent_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"

payload = {
    "description": "This is the latest description of data_agent.",
    "data_analytics_agent": {
        "published_context": {
            "datasource_references": bigquery_data_sources,
            "system_instruction": system_instruction
        }
    },
}

fields = ["description", "data_analytics_agent"]
params = {
    "updateMask": ",".join(fields)
}

data_agent_response = requests.patch(
    data_agent_url, headers=headers, params=params, json=payload
)

if data_agent_response.status_code == 200:
    print("Data Agent updated successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error Updating Data Agent: {data_agent_response.status_code}")
    print(data_agent_response.text)

In [ ]:
# @title Set IAM Policy for Data Agent


''' PLEASE NOTE THIS API CALLS OVERRIDES EXISTING PERMISSION FOR THE RESOURCE.
For preserving existing policy in practise call Get IAM policy to fetch existing policy and pass it along with additional changes
in the call to Set IAM Policy
'''
data_agent_id = "data_agent_1" # @param {type:"string"}
users = "abc@google.com, aabbcc@gmail.com" # @param {type:"string"}

data_agent_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}:setIamPolicy"


# Request body
payload = {
    "policy": {
        "bindings": [
            {
                "role": "roles/geminidataanalytics.dataAgentEditor",
                "members": [
                    f"user:{i.strip()}" for i in users.split(",")
                ]
            }
        ]
    }
}

data_agent_response = requests.post(
    data_agent_url, headers=headers, json=payload
)

if data_agent_response.status_code == 200:
    print("IAM Policy set successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error setting IAM policy: {data_agent_response.status_code}")
    print(data_agent_response.text)

In [ ]:
# @title Get IAM Policy for Data Agent

data_agent_id = "data_agent_1" # @param {type:"string"}

data_agent_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}:getIamPolicy"

# Request body
payload = {
  "resource": f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"
}

data_agent_response = requests.post(
    data_agent_url, headers=headers, json=payload
)

if data_agent_response.status_code == 200:
    print("IAM Policy fetched successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error fetching IAM policy: {data_agent_response.status_code}")
    print(data_agent_response.text)

In [ ]:
# @title [Soft Delete] Delete Data Agent

data_agent_id = "data_agent_1" # @param {type:"string"}

data_agent_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"

data_agent_response = requests.delete(
    data_agent_url, headers=headers
)

if data_agent_response.status_code == 200:
    print("Data Agent deleted successfully!")
    print(json.dumps(data_agent_response.json(), indent=2))
else:
    print(f"Error Deleting Data Agent: {data_agent_response.status_code}")
    print(data_agent_response.text)

# Data Chat Service

In [ ]:
# @title Create Conversation

data_agent_id = "data_agent_1" # @param {type:"string"}
conversation_id = "conversation_1" # @param {type:"string"}

conversation_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/conversations"

conversation_payload = {
    "agents": [
        f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"
    ],
    "name": f"projects/{billing_project}/locations/{location}/conversations/{conversation_id}"
}
params = {
    "conversation_id": conversation_id
}

conversation_response = requests.post(conversation_url, headers=headers, params=params, json=conversation_payload)

if conversation_response.status_code == 200:
    print("Conversation created successfully!")
    print(json.dumps(conversation_response.json(), indent=2))
else:
    print(f"Error creating Conversation: {conversation_response.status_code}")
    print(conversation_response.text)

In [ ]:
# @title Get Conversation

conversation_id = "conversation_1" # @param {type:"string"}

conversation_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/conversations/{conversation_id}"

conversation_response = requests.get(conversation_url, headers=headers)

# Handle the response
if conversation_response.status_code == 200:
    print("Conversation fetched successfully!")
    print(json.dumps(conversation_response.json(), indent=2))
else:
    print(f"Error while fetching conversation: {conversation_response.status_code}")
    print(conversation_response.text)


In [ ]:
# @title List Conversation

conversation_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/conversations"


conversation_response = requests.get(conversation_url, headers=headers)

# Handle the response
if conversation_response.status_code == 200:
    print("Conversation fetched successfully!")
    print(json.dumps(conversation_response.json(), indent=2))
else:
    print(f"Error while fetching conversation: {conversation_response.status_code}")
    print(conversation_response.text)

In [ ]:
# @title Streaming Chat Messages

def is_json(str):
 try:
     json_object = json_lib.loads(str)
 except ValueError as e:
     return False
 return True

def handle_text_response(resp):
 parts = resp['parts']
 print(''.join(parts))

def get_property(data, field_name, default = ''):
 return data[field_name] if field_name in data else default

def display_schema(data):
 fields = data['fields']
 df = pd.DataFrame({
   "Column": map(lambda field: get_property(field, 'name'), fields),
   "Type": map(lambda field: get_property(field, 'type'), fields),
   "Description": map(lambda field: get_property(field, 'description', '-'), fields),
   "Mode": map(lambda field: get_property(field, 'mode'), fields)
 })
 display(df)

def display_section_title(text):
 display(HTML('<h2>{}</h2>'.format(text)))

def format_bq_table_ref(table_ref):
 return '{}.{}.{}'.format(table_ref['projectId'], table_ref['datasetId'], table_ref['tableId'])

def format_looker_table_ref(table_ref):
 return 'lookmlModel: {}, explore: {}, lookerInstanceUri: {}'.format(table_ref['lookmlModel'], table_ref['explore'], table_ref['lookerInstanceUri'])

def display_datasource(datasource):
 source_name = ''

 if 'studioDatasourceId' in datasource:
   source_name = datasource['studioDatasourceId']
 elif 'lookerExploreReference' in datasource:
   source_name = format_looker_table_ref(datasource['lookerExploreReference'])
 else:
   source_name = format_bq_table_ref(datasource['bigqueryTableReference'])

 print(source_name)
 display_schema(datasource['schema'])

def handle_schema_response(resp):
 if 'query' in resp:
   print(resp['query']['question'])
 elif 'result' in resp:
   display_section_title('Schema resolved')
   print('Data sources:')
   for datasource in resp['result']['datasources']:
     display_datasource(datasource)

def handle_data_response(resp):
 if 'query' in resp:
   query = resp['query']
   display_section_title('Retrieval query')
   print('Query name: {}'.format(query['name']))
   print('Question: {}'.format(query['question']))
   print('Data sources:')
   for datasource in query['datasources']:
     display_datasource(datasource)
 elif 'generatedSql' in resp:
   display_section_title('SQL generated')
   print(resp['generatedSql'])
 elif 'result' in resp:
   display_section_title('Data retrieved')

   fields = map(lambda field: get_property(field, 'name'), resp['result']['schema']['fields'])
   dict = {}

   for field in fields:
     dict[field] = map(lambda el: get_property(el, field), resp['result']['data'])

   display(pd.DataFrame(dict))

def handle_chart_response(resp):
 if 'query' in resp:
   print(resp['query']['instructions'])
 elif 'result' in resp:
   vegaConfig = resp['result']['vegaConfig']
   alt.Chart.from_json(json_lib.dumps(vegaConfig)).display();

def handle_error(resp):
 display_section_title('Error')
 print('Code: {}'.format(resp['code']))
 print('Message: {}'.format(resp['message']))

def get_stream(url, json):
 s = requests.Session()

 acc = ''

 with s.post(url, json=json, headers=headers, stream=True) as resp:
   for line in resp.iter_lines():
     if not line:
       continue

     decoded_line = str(line, encoding='utf-8')

     if decoded_line == '[{':
       acc = '{'
     elif decoded_line == '}]':
       acc += '}'
     elif decoded_line == ',':
       continue
     else:
       acc += decoded_line

     if not is_json(acc):
       continue

     data_json = json_lib.loads(acc)

     if not 'systemMessage' in data_json:
       if 'error' in data_json:
         handle_error(data_json['error'])
       continue

     if 'text' in data_json['systemMessage']:
       handle_text_response(data_json['systemMessage']['text'])
     elif 'schema' in data_json['systemMessage']:
       handle_schema_response(data_json['systemMessage']['schema'])
     elif 'data' in data_json['systemMessage']:
       handle_data_response(data_json['systemMessage']['data'])
     elif 'chart' in data_json['systemMessage']:
       handle_chart_response(data_json['systemMessage']['chart'])
     else:
       colored_json = highlight(acc, lexers.JsonLexer(), formatters.TerminalFormatter())
       print(colored_json)
     print('\n')
     acc = ''


In [ ]:
# @title [Stateful] Chat using Conversation

data_agent_id = "data_agent_1" # @param {type:"string"}
conversation_id = "conversation_1" # @param {type:"string"}

chat_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}:chat"

# Construct the payload
chat_payload = {
    "parent": f"projects/{billing_project}/locations/global",
    "messages": [
        {
            "userMessage": {
                "text": "Make a bar graph for the top 5 states by the total number of airports"
            }
        }
    ],
    "conversation_reference": {
        "conversation": f"projects/{billing_project}/locations/{location}/conversations/{conversation_id}",
        "data_agent_context": {
            "data_agent": f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}",
            # "credentials": looker_credentials #Uncomment this if your dataAgent contains looker as dataSource
        }
    }
}

# Call get_stream method to stream the response
get_stream(chat_url, chat_payload)


top 5 states by the total number of airports




Data sources:
bigquery-public-data.faa.us_airports


,Column,Type,Description,Mode
0,object_id,STRING,-,NULLABLE
1,global_id,STRING,-,NULLABLE
2,faa_identifier,STRING,-,NULLABLE
3,name,STRING,-,NULLABLE
4,latitude,FLOAT64,-,NULLABLE
5,longitude,FLOAT64,-,NULLABLE
6,airport_geom,GEOGRAPHY,-,NULLABLE
7,elevation,FLOAT64,-,NULLABLE
8,icao_id,STRING,-,NULLABLE
9,airport_type,STRING,-,NULLABLE


Query name: top_5_states_by_airport_count
Question: What are the top 5 states by the number of airports?
Data sources:
bigquery-public-data.faa.us_airports


,Column,Type,Description,Mode
0,object_id,STRING,-,NULLABLE
1,global_id,STRING,-,NULLABLE
2,faa_identifier,STRING,-,NULLABLE
3,name,STRING,-,NULLABLE
4,latitude,FLOAT64,-,NULLABLE
5,longitude,FLOAT64,-,NULLABLE
6,airport_geom,GEOGRAPHY,-,NULLABLE
7,elevation,FLOAT64,-,NULLABLE
8,icao_id,STRING,-,NULLABLE
9,airport_type,STRING,-,NULLABLE


SELECT state_abbreviation, COUNT(*) AS num_airports
FROM `bigquery-public-data`.`faa`.`us_airports`
GROUP BY state_abbreviation
ORDER BY num_airports DESC
LIMIT 5;






,state_abbreviation,num_airports
0,2033,2033
1,886,886
2,870,870
3,762,762
4,759,759




Create a bar chart showing the top 5 states and the number of airports in each state. The state should be on the x axis and the number of airports on the y axis.




alt.Chart(...)



Here is a bar chart of the top 5 states by number of airports. TX has the most airports with 2033, followed by FL with 886, CA with 870, AK with 762, and PA with 759.




In [ ]:
# @title [Stateless] Chat using Data Agent

data_agent_id = "data_agent_1" # @param {type:"string"}

chat_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}:chat"

# Construct the payload
chat_payload = {
    "parent": f"projects/{billing_project}/locations/global",
    "messages": [
        {
            "userMessage": {
                "text": "Make a bar graph for the top 5 states by the total number of airports"
            }
        }
    ],
    "data_agent_context": {
        "data_agent": f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}",
        # "credentials": looker_credentials #Uncomment this if your dataAgent contains looker as dataSource
    }
}

# Call get_stream method to stream the response
get_stream(chat_url, chat_payload)


In [ ]:
# @title [Stateless] Chat using Inline Context

chat_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/global:chat"

# Construct the payload
chat_payload = {
    "parent": f"projects/{billing_project}/locations/global",
    "messages": [
        {
            "userMessage": {
                "text": "Show the schema of datasource"
            }
        }
    ],
    "inline_context": {
        "datasource_references": bigquery_data_sources,
        "options": {
            "analysis": {
                # Optional - if wanting to use advanced analysis with python
                "python": {
                    "enabled": True
                }
            }
        }
    }
}

# Call get_stream method to stream the response
get_stream(chat_url, chat_payload)


In [ ]:
# @title List Messages

conversation_id = "conversation_21" # @param {type:"string"}

conversation_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/{location}/conversations/{conversation_id}/messages"

conversation_response = requests.get(conversation_url, headers=headers)

# Handle the response
if conversation_response.status_code == 200:
    print("Conversation fetched successfully!")
    print(json.dumps(conversation_response.json(), indent=2))
else:
    print(f"Error while fetching conversation: {conversation_response.status_code}")
    print(conversation_response.text)


## Stateless Multi-turn Conversation

In [ ]:
# @title get_stream function for multi-turn conversation

# NOTE: This methods is same as get_stream() method present in Streaming Chat Messages section.
# The only difference is that - here we are storing the response in an array "conversation_messages" to save the conversation.


def get_stream_multi_turn(url, json, conversation_messages):
 s = requests.Session()

 acc = ''

 with s.post(url, json=json, headers=headers, stream=True) as resp:
   for line in resp.iter_lines():
     if not line:
       continue

     decoded_line = str(line, encoding='utf-8')

     if decoded_line == '[{':
       acc = '{'
     elif decoded_line == '}]':
       acc += '}'
     elif decoded_line == ',':
       continue
     else:
       acc += decoded_line

     if not is_json(acc):
       continue

     data_json = json_lib.loads(acc)
     # Store the response to be used in next iteration.
     conversation_messages.append(data_json)

     if not 'systemMessage' in data_json:
       if 'error' in data_json:
         handle_error(data_json['error'])
       continue

     if 'text' in data_json['systemMessage']:
       handle_text_response(data_json['systemMessage']['text'])
     elif 'schema' in data_json['systemMessage']:
       handle_schema_response(data_json['systemMessage']['schema'])
     elif 'data' in data_json['systemMessage']:
       handle_data_response(data_json['systemMessage']['data'])
     elif 'chart' in data_json['systemMessage']:
       handle_chart_response(data_json['systemMessage']['chart'])
     else:
       colored_json = highlight(acc, lexers.JsonLexer(), formatters.TerminalFormatter())
       print(colored_json)
     print('\n')
     acc = ''


In [ ]:
# @title Multi-turn Conversation helper function

chat_url = f"{base_url}/v1alpha/projects/{billing_project}/locations/global:chat"

# Re-used across requests to track previous turns
conversation_messages = []

data_agent_id = "data_agent_1" # @param {type:"string"}

# Helper function for calling the API
def multi_turn_conversation(msg):

  userMessage = {
      "userMessage": {
          "text": msg
      }
  }

  # Send multi-turn request by including previous turns, plus new message
  conversation_messages.append(userMessage)

  # Construct the payload
  chat_payload = {
      "parent": f"projects/{billing_project}/locations/global",
      "messages": conversation_messages,
      "data_agent_context": {
          "data_agent": f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}",
          # "credentials": looker_credentials # Uncomment this if your dataAgent contains looker as dataSource
      }

      # uncomment the lines below when using chat with inline_context
      # "inline_context": {
      #     "datasource_references": bigquery_data_sources,
      # }
  }

  # Call get_stream_multi_turn method to stream the response
  get_stream_multi_turn(chat_url, chat_payload, conversation_messages)


In [ ]:
# Send first-turn request
multi_turn_conversation("List of the top 5 states by the total number of airports")

In [ ]:
# Send follow-up-turn request
multi_turn_conversation("Can you please make it a pie chart?")